# LSTM 模型

     数据结构说明：
     
     1、基本数据
        1） 
        self: Open, Close, ..., Volume , INCOME, Flag   -> 为防止某些数据过大， 统一下做log，后面其它的数据统一采用该种方式。
        lstm data : line 1: [[open,....],[open,...]] 连续N个数据，通过LSTM 连续观察特征值，再进行分类
        
     2、网结构：
        
        1）
        Seq: 
            B, N , (O,H,L,..)  
            lstm --> B, N , 256  -->  liner --> B, 128  --> B,  64 --> B, 3
    

# 准备数据

In [1]:
from stock.common.sdata import StockData
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
code = '600016'
sd = StockData(code)
data = sd.combine_income(ndays=5)
data_in , target = sd.package_data(data=data)
data_in = [np.log(x.values) for x in data_in]

dlen = int(len(data_in) * 0.9)
train_data, train_target = torch.from_numpy(np.array(data_in[0:dlen])).float(), torch.from_numpy(np.array(target[0:dlen])).long()
test_data, test_target = torch.from_numpy(np.array(data_in[dlen:])).float(), torch.from_numpy(np.array(target[dlen:]).squeeze()).long()


In [3]:
print(test_target)

tensor([1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2,
        2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1,
        1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1,
        2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 2,
        2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2,
        2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 0, 2, 2, 2, 2, 2, 0,
        1, 1, 1, 0, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 1, 0, 0, 2, 2,
        2, 2, 2, 1, 0, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 0, 1, 1, 0, 1, 2, 2, 2,
        2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1])


# 网络模型 一

In [9]:
# 模型
import torch.nn  as nn
class Net(nn.Module):
    def __init__(self, n_feature, n_hidden=256, n_target=3):
        super(Net,self).__init__()
        self.lstm = nn.LSTM(input_size=n_feature,hidden_size=n_hidden,batch_first=True)
        self.layer = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, n_target),
            nn.ReLU()
        )
    
    def forward(self, data):
        x, _ = self.lstm(data)
        x = self.layer(x)
        x = x[:,-1,:]
        return x

In [19]:
print(test_data.size())
out = net(test_data)
pred = torch.max(F.softmax(out), 1)[1]
print(pred)

torch.Size([286, 5, 6])
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])


/home/hecong/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# 训练网络
import torch.optim as optim
import torch.nn.functional as F
import os
import copy
from tensorboardX import SummaryWriter

#   tensorboard --logdir /home/hecong/dplearn/base/log

writer = SummaryWriter("/home/hecong/dplearn/base/log")
feature = train_data.shape[2]
dataset = TensorDataset(train_data, train_target)
train_loader = DataLoader(dataset=dataset, batch_size=64, shuffle=True)
net = Net(n_feature=feature)
if os.path.exists('net_model.pkl'):
    print('加载上次训练模型...')
    net.load_state_dict(torch.load('net_model.pkl'))
    

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=1e-3)


epcho = 10000
for epcho in range(epcho):
    run_loss = 0
    for step, (x,y) in enumerate(train_loader):
        net.train()
        optimizer.zero_grad()
        target = y.view(-1)
        out = net(x)
#         print(out)
#         pred = torch.max(F.softmax(out), 1)[1]
#         print(pred)
#         print(target)
        loss = loss_function(out,target)
        run_loss += loss
        loss.backward()
        optimizer.step()
    if epcho % 5 == 0:
        
        print('epcho {} step {} loss {:.4f}'.format(epcho,step, run_loss/step))
        net.eval()
        validate_out = net(test_data)
        pred = torch.max(F.softmax(validate_out), 1)[1]
        correct = sum(pred.data.numpy() == test_target.data.numpy())
        print('correct number : %s (%s)' % (correct, (correct / len(test_target.data.numpy()))))
        _wts = copy.deepcopy(net.state_dict())  
        torch.save(_wts,'net_model.pkl')       
        
        for name, param in net.named_parameters():
            writer.add_histogram(name, param.clone().cpu().data.numpy(), epcho)        
        
        
        
    

epcho 0 step 40 loss 1.0037
correct number : 116 (0.40559440559440557)


/home/hecong/venv/lib/python3.6/site-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epcho 5 step 40 loss 0.9713
correct number : 116 (0.40559440559440557)
epcho 10 step 40 loss 0.9768
correct number : 116 (0.40559440559440557)
epcho 15 step 40 loss 0.9673
correct number : 116 (0.40559440559440557)
epcho 20 step 40 loss 0.9707
correct number : 116 (0.40559440559440557)
epcho 25 step 40 loss 0.9660
correct number : 116 (0.40559440559440557)
epcho 30 step 40 loss 0.9913
correct number : 116 (0.40559440559440557)
epcho 35 step 40 loss 0.9782
correct number : 116 (0.40559440559440557)
epcho 40 step 40 loss 0.9643
correct number : 116 (0.40559440559440557)
epcho 45 step 40 loss 0.9642
correct number : 116 (0.40559440559440557)
epcho 50 step 40 loss 0.9643
correct number : 116 (0.40559440559440557)
epcho 55 step 40 loss 0.9724
correct number : 116 (0.40559440559440557)
epcho 60 step 40 loss 0.9784
correct number : 116 (0.40559440559440557)
epcho 65 step 40 loss 0.9659
correct number : 116 (0.40559440559440557)
epcho 70 step 40 loss 0.9634
correct number : 116 (0.405594405594

In [12]:
writer.close()

In [10]:
# dd = iter(train_loader)
# x, y = next(dd)
# out = net(x)
# print(out.size())

print(train_data.size())


torch.Size([2566, 5, 6])
